In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("taxi").getOrCreate()

24/12/10 15:50:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
yellow202101 = spark.read.format('csv').load('data/yellow_tripdata_2021-01.csv', inferSchema=True, header=True)
yellow202102 = spark.read.format('csv').load('data/yellow_tripdata_2021-02.csv', inferSchema=True, header=True)
yellow202103 = spark.read.format('csv').load('data/yellow_tripdata_2021-03.csv', inferSchema=True, header=True)
yellow202104 = spark.read.format('csv').load('data/yellow_tripdata_2021-04.csv', inferSchema=True, header=True)
yellow202105 = spark.read.format('csv').load('data/yellow_tripdata_2021-05.csv', inferSchema=True, header=True)
yellow202106 = spark.read.format('csv').load('data/yellow_tripdata_2021-06.csv', inferSchema=True, header=True)
yellow202107 = spark.read.format('csv').load('data/yellow_tripdata_2021-07.csv', inferSchema=True, header=True)

In [13]:
zone_df = spark.read.format('csv').load('data/taxi+_zone_lookup.csv', inferSchema=True, header=True)

In [14]:
zone_df.show(20)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [11]:
trips_df = yellow202101.union(yellow202102).union(yellow202103).union(yellow202104).union(yellow202105).union(yellow202106).union(yellow202107)

In [12]:
trips_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [15]:
trips_df.createOrReplaceTempView('trips')
zone_df.createOrReplaceTempView('zone')

In [33]:
#분석에 필요할 컬럼들만 따로 빼서 출력
query = '''
select
t.VendorID,
TO_DATE(t.tpep_pickup_datetime) as pickup_date,
TO_DATE(t.tpep_dropoff_datetime) as dropoff_date,
HOUR(t.tpep_pickup_datetime)  as pickup_time,
HOUR(t.tpep_dropoff_datetime) as dropoff_time,
t.passenger_count,
t.trip_distance,
t.tip_amount,
t.total_amount,
t.payment_type,
pz.Zone as pickup_zone,
dz.Zone as dropoff_zone
from trips t
LEFT JOIN zone pz ON t.PULocationID = pz.LocationID
LEFT JOIN zone dz ON t.DOLocationID = dz.LocationID
'''

In [34]:
comb_df = spark.sql(query)

In [23]:
comb_df.count() #1500만건의 데이터...

15000700

In [35]:
comb_df.show(5)

+--------+-----------+------------+-----------+------------+---------------+-------------+----------+------------+------------+--------------------+----------------+
|VendorID|pickup_date|dropoff_date|pickup_time|dropoff_time|passenger_count|trip_distance|tip_amount|total_amount|payment_type|         pickup_zone|    dropoff_zone|
+--------+-----------+------------+-----------+------------+---------------+-------------+----------+------------+------------+--------------------+----------------+
|       1| 2021-01-01|  2021-01-01|          0|           0|              1|          2.1|       0.0|        11.8|           2| Lincoln Square East|    Central Park|
|       1| 2021-01-01|  2021-01-01|          0|           0|              1|          0.2|       0.0|         4.3|           2|Upper West Side N...|Manhattan Valley|
|       1| 2021-01-01|  2021-01-01|          0|           1|              1|         14.7|      8.65|       51.95|           1|         JFK Airport|         Midwood|
|   

In [36]:
comb_df.createOrReplaceTempView('comb')

query = '''
select pickup_date, pickup_time 
from comb 
where pickup_time>0
'''

spark.sql(query).show()

+-----------+-----------+
|pickup_date|pickup_time|
+-----------+-----------+
| 2020-12-31|         23|
| 2020-12-31|         21|
| 2020-12-31|         23|
| 2020-12-31|         18|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2020-12-31|         23|
| 2020-12-31|         13|
| 2020-12-31|         23|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
+-----------+-----------+
only showing top 20 rows



In [37]:
query = '''
select pickup_date, pickup_time 
from comb 
where pickup_date<'2020-12-31'
'''
spark.sql(query).show()

+-----------+-----------+
|pickup_date|pickup_time|
+-----------+-----------+
| 2009-01-01|          0|
| 2008-12-31|         23|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2008-12-31|         23|
| 2008-12-31|         23|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          1|
| 2009-01-01|         17|
| 2009-01-01|         17|
| 2009-01-01|         18|
| 2009-01-01|          1|
| 2009-01-01|          1|
| 2009-01-01|          2|
| 2009-01-01|          2|
| 2009-01-01|          0|
+-----------+-----------+
only showing top 20 rows



In [38]:
comb_df.describe()

DataFrame[summary: string, VendorID: string, pickup_time: string, dropoff_time: string, passenger_count: string, trip_distance: string, tip_amount: string, total_amount: string, payment_type: string, pickup_zone: string, dropoff_zone: string]

In [ ]:
spark.sql(query).explain()

In [40]:
query1 = '''
select pickup_date, pickup_time
from comb
where pickup_date < '2020-12-31'
'''

spark.sql(query1).show()


+-----------+-----------+
|pickup_date|pickup_time|
+-----------+-----------+
| 2009-01-01|          0|
| 2008-12-31|         23|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2008-12-31|         23|
| 2008-12-31|         23|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          1|
| 2009-01-01|         17|
| 2009-01-01|         17|
| 2009-01-01|         18|
| 2009-01-01|          1|
| 2009-01-01|          1|
| 2009-01-01|          2|
| 2009-01-01|          2|
| 2009-01-01|          0|
+-----------+-----------+
only showing top 20 rows



In [46]:
# 실행 계획, 실행 결과(4040에서 확인해보기)
query2 = '''
select pickup_date, pickup_time
from comb
where pickup_time > 0 and pickup_time<=12
'''

spark.sql(query2).show()

+-----------+-----------+
|pickup_date|pickup_time|
+-----------+-----------+
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
| 2021-01-01|          1|
+-----------+-----------+
only showing top 20 rows



In [49]:
query3 = '''
select 
    pickup_date, 
    first(pickup_time) as pickup_time
from comb
where pickup_time > 0
group by pickup_date
order by pickup_date
'''

spark.sql(query3).explain()

== Physical Plan ==
*(12) Sort [pickup_date#992 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(pickup_date#992 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#2535]
   +- *(11) HashAggregate(keys=[pickup_date#992], functions=[first(pickup_time#994, false)])
      +- Exchange hashpartitioning(pickup_date#992, 200), ENSURE_REQUIREMENTS, [id=#2531]
         +- *(10) HashAggregate(keys=[pickup_date#992], functions=[partial_first(pickup_time#994, false)])
            +- *(10) Project [cast(tpep_pickup_datetime#41 as date) AS pickup_date#992, hour(cast(tpep_pickup_datetime#41 as timestamp), Some(Asia/Seoul)) AS pickup_time#994]
               +- *(10) BroadcastHashJoin [DOLocationID#48], [LocationID#998], LeftOuter, BuildRight, false
                  :- *(10) Project [tpep_pickup_datetime#41, DOLocationID#48]
                  :  +- *(10) BroadcastHashJoin [PULocationID#47], [LocationID#666], LeftOuter, BuildRight, false
                  :     :- Union
                  :     :  

In [ ]:
spark.stop()